# Select Rental Offices around CNU

### Read data

In [1]:
import pandas as pd
import os

rental = os.path.join('preprocessed', 'season2_rental_office.csv')
dpath = os.path.join('preprocessed', '2022_season2_processed.csv')
df = pd.read_csv(dpath,
                 index_col=0,
                 parse_dates=['대여일시', '반납일시'],
                 infer_datetime_format=True)
df_rental = pd.read_csv(rental,
                        index_col=0)
df_rental.head()

,구분,대여소명,구,동,위도,경도
번호,,,,,,
1,타슈2,타슈관제센터,유성구,문지동,36.389501,127.409377
3,타슈2,탄방동 한사랑병원,서구,탄방동,36.348446,127.390052
4,타슈2,한가람아파트 2동,서구,탄방동,36.348201,127.390409
5,타슈2,목련아파트 103동,서구,탄방동,36.350037,127.390402
6,타슈2,크로바아파트 103동,서구,둔산동,36.352480,127.390385


### Filter by location

충남대 좌표 = [36.3687979, 127.3468559]

적어도 유성온천역~월드컵대로 까지는 포함할 수 있도록 좌표를 선정

월드컵대로 아래쪽 좌표 = [36.3476, 127.3468]

In [2]:
36.3687979-36.3476

0.021197899999997105

충남대를 중심으로 위도/경도 +-0.0212까지 포함하는 영역내의 대여소 및 데이터 필터링

In [3]:
pos_cnu = [36.3687979, 127.3468559]
offset = 0.0212
left_top = list(map(lambda x: x-offset, pos_cnu))
right_bottom = list(map(lambda x: x+offset, pos_cnu))
print(left_top, right_bottom)

[36.3475979, 127.3256559] [36.3899979, 127.36805589999999]


In [4]:
temp = df_rental
temp = temp.loc[(temp['위도'] >= left_top[0]) & (temp['경도'] >= left_top[1])]
temp = temp.loc[(temp['위도'] <= right_bottom[0]) & (temp['경도'] <= right_bottom[1])]
temp

,구분,대여소명,구,동,위도,경도
번호,,,,,,
18,타슈2,다모아아파트 뒤 육교밑,서구,월평동,36.363085,127.366753
19,타슈2,은평공원 내 테니스장 입구,서구,월평동,36.358670,127.363647
22,타슈2,유성한가족 요양병원 소공원,유성구,봉명동,36.355079,127.350214
85,타슈2,충대정문오거리2(정문)(56),유성구,궁동,36.361611,127.345168
96,타슈2,월평역 1번출구,서구,월평동,36.357998,127.364775
...,...,...,...,...,...,...
1084,타슈2,유성온천역 3번출구,유성구,봉명동,36.352996,127.340988
1085,타슈2,유성온천역 6번출구,유성구,봉명동,36.354255,127.341439
1142,타슈2,유성생명과학고(64),유성구,상대동,36.348285,127.330635


In [5]:
# temp2 = pd.concat([temp, temp], axis=0)
# temp2 = temp2.drop_duplicates()
# temp2
import numpy as np
def retain_from_index_array(target: pd.Series, index: np.ndarray) -> pd.Series:
    c = target.copy()
    for i, v in target.items():
        if v in index:      # if index array contains v
            c[i] = True     # retain this row.
        else:               # if not contains v
            c[i] = False    # remove this row.
    return c
index_arr = temp.index
df_temp1 = df.loc[retain_from_index_array(df['대여스테이션'], index_arr)]
df_temp2 = df.loc[retain_from_index_array(df['반납스테이션'], index_arr)]
df_temp = pd.concat([df_temp1, df_temp2])
print(f'df_temp1: {df_temp1.shape[0]}, df_temp2: {df_temp2.shape[0]}')
df_temp = df_temp.drop_duplicates()
df_temp

df_temp1: 180743, df_temp2: 180715


,대여스테이션,대여일시,반납스테이션,반납일시,이동거리,rental_time
119,191,2022-07-25 16:33:58,413,2022-07-25 16:58:04,4000,1446
120,191,2022-07-25 16:34:40,413,2022-07-25 16:58:19,4000,1419
121,191,2022-07-25 16:34:48,413,2022-07-25 16:58:22,4000,1414
127,770,2022-07-25 17:18:12,202,2022-07-25 21:40:12,1000,15720
141,191,2022-07-25 18:00:37,444,2022-07-25 19:10:39,5500,4202
...,...,...,...,...,...,...
1330029,777,2022-12-31 22:38:40,768,2022-12-31 22:50:36,1100,716
1330043,708,2022-12-31 22:41:34,769,2022-12-31 23:04:36,3100,1382
1330078,455,2022-12-31 22:57:37,330,2022-12-31 23:43:18,5300,2741
1330101,414,2022-12-31 23:04:07,1172,2022-12-31 23:27:07,2800,1380


### Save results

In [6]:
output_path = os.path.join('postprocessed', 'data_around_cnu.csv')
rental_output_path = os.path.join('postprocessed', 'rental_around_cnu.csv')
df_temp.to_csv(output_path)
temp.to_csv(rental_output_path)